Copyright (C) 2020  by Karim AÏT AMMAR, Enguerrand LUCAS, Julie DIANI 

# Homogenization - main

This software aims at providing a user-friendly tool for the homogenization of a matrix containing fillers. 
Our main goals were:
- Compute the linear elastic and linear viscoelastic behaviors (see homogenization_visco) of microstructures defined by geometry and constitutive phases behaviors,
- Compare models for a given microstructure,
- Offer an inverse method to reach microstructure parameters such as filler volume fraction, one of the constitutive phase mechanical properties when knowing the target homogeneous behavior
- Provide comprehensive descriptions of the implemented models emphasizing their relevance and llimitation. 
- Provide with an evolutive tool that anyone could increment with other models or microstructures

This code contains multiple sections. 

**When the script is run for the first time on a new kernel, the following sections have to be executed according to this order**:
- I- Classes and modules import
- II- Useful functions

The other sections are independant.

The first time ever you are using this notebook please compile the fortran tools in a shell by typing:

$ f2py -c fortran_tools.f -m fortran_tools

---

## I- Classes and modules import

In [1]:
!pip3 install ipywidgets
from IPython.display import clear_output, display, Markdown
import ipywidgets as widgets
from classes_v0 import * 
from os import listdir
import pandas as pd
import csv

clear_output()
%matplotlib widget
print("Package downloaded")
print("Imported models : {}".format(list_models))

Package downloaded
Imported models : [<class 'classes_v0.Voigt_Bound'>, <class 'classes_v0.Reuss_Bound'>, <class 'classes_v0.Mori_Tanaka'>, <class 'classes_v0.Differential_Scheme'>, <class 'classes_v0.Self_Consistent_Hill'>, <class 'classes_v0.Self_Consistent_III'>, <class 'classes_v0.Self_Consistent_IV'>]


## II- Useful functions

In [2]:
parameters_name = {'K': 'Bulk modulus K',
                   'G': 'Shear modulus G',
                   'E': 'Young modulus E',
                   'nu': "Poisson's ratio " + r'$\nu$',
                   'C': "Stiffness matrix C",
                   'S': "Compliance matrix S"}
parameters_name_bis = {value: key for (key, value) in parameters_name.items()}
layout={'border': '1px solid #FF425BF5'}

def gen_tab_behavior(anisotropic=True):
    """
    Generate a 'tab' widget defining the behavior parameters.
    Each generated tab refers to a behavior from the 'dict_behaviors' dictionnary defined in 'classes.py'.
    The 'isotropic' parameter is 'True' if one only wants to display isotropic behaviors.
    The function returns:
    - A list of list of widgets 'list_widgets' with as many lists as tabs. Each list contains the widgets for the parameters associated with the behavior of the corresponding tab.
    - The tab widget to be displayed.
    """
    behaviors_str = list(dict_behaviors.keys()) # Lists the names of the behaviors implemented in classes.py
    list_widgets = [] # List of lists, each list refers to a tab of the final widget and contains the non-formatted widgets of this tab
    tab_titles = [] # Tabs names, each tab refers to a behavior type ('Isotropic K and G', 'Anisotropic', etc..) 
    # Generation of the widgets for each tab
    for behavior_str in behaviors_str:
        if 'Anisotropic' in behavior_str and not anisotropic:
            continue
        widgets_onglet = []
        parameters = dict_behaviors[behavior_str] # Behavior-associated parameters (example : ['K', 'G'] for Isotropic)
        for parameter in parameters:
            w = widgets.BoundedFloatText(value=200, min=0, max=10**12, step=1) # 'parameter' associated widget
            if parameter == 'nu':
                w.max = 0.5
                w.value = 0.3
                w.step = 0.1
            if 'Anisotropic' in behavior_str:
                # Recovery of the anisotropic behaviors files
                input_folder = 'inputs/anisotropic_behaviors'
                behavior_files = listdir(input_folder)
                # Selection of .csv and .txt files
                behavior_files = [file for file in behavior_files if (file.endswith('.txt') or file.endswith('.csv'))]
                # Widget creation
                w = widgets.Dropdown(options=behavior_files)
            w_label = widgets.Label(value=parameters_name[parameter])
            widgets_onglet.append(widgets.HBox([w_label, w]))
        # Input files description
        if 'Anisotropic' in behavior_str:
            w_label = widgets.Label(value="Entry file must be a csv file with 6 rows and columns of numbers.\nInput folder: inputs/anisotropic_behaviors")
            widgets_onglet.append(w_label)
        list_widgets.append(widgets_onglet)
        tab_titles.append(behavior_str)
    # Tab creation
    tab = widgets.Tab()
    tab.children = [widgets.VBox(w) for w in list_widgets]
    for pos, title in enumerate(tab_titles):
        tab.set_title(pos, title)
    return list_widgets, tab

def read_behavior(tab, list_widgets):
    """
    Reads tab widgets created by the 'gen_tab_behavior', returns a 'behavior' dictionnary.
    """
    behavior_int = tab.selected_index # Index of the tab selected by the user
    behavior_str = tab.get_title(behavior_int)
    widgets_parameters = list_widgets[behavior_int] # Parameter widgets of the active tab
    if 'Isotropic' in behavior_str:
        # Isotropic behaviors
        behavior = {parameters_name_bis[w.children[0].value]: w.children[1].value for w in widgets_parameters}
    else:
        # Anisotropic behaviors
        w_parameter = widgets_parameters[0] # HBox widget containing a label widget and a text widget
        parameter = parameters_name_bis[w_parameter.children[0].value] # Name of the parameter
        file_name = "inputs/anisotropic_behaviors/" + w_parameter.children[1].value
        # Anisotropic behavior file reading
        value = []
        with open(file_name) as csvfile:
            reader = csv.reader(csvfile, quoting=csv.QUOTE_NONNUMERIC) # change contents to floats
            for row in reader: # each row is a list
                value.append(row)
        # Generation of the 'behavior' dict
        behavior = {parameter: np.array(value)}
    return behavior

def gen_tab_type():
    """
    Creates a tab widget setting the inclusion geometric parameters (aspects ratio, orientation, etc..).
    TODO : Inclure l'orientation
    Each tab refers to a key of the 'dict_types' dictionnary defined in 'classes.py'.
    Returns:
    - a list of list of widgets 'list_widgets' with as many lists as tabs. Each list contains widgets associated to a type.
    - a tab widget to be displayed.
    """
    list_widgets = [] # List of lists, each list refers to a tab and contains this tab widgets
    # Tab per tab widgets generation
    for type_int in dict_types.keys():
        if type_int == 0:
            # Spheres, additional parameters are not needed
            list_widgets.append([])
        else:
            # Initialisation
            list_widgets.append([])
            # Ellipsoids
            for n in ['1', '2']:
                w_label = widgets.Label(value="Aspect ratio " + str(n))
                w_aspect_ratio = widgets.BoundedFloatText(value=0.5, min=0.0001, max=10000, step=0.5)
                list_widgets[-1] += [widgets.HBox([w_label, w_aspect_ratio])]
    # Tab creation
    tab = widgets.Tab()
    # Tabs attribution
    tab.children = [widgets.HBox(w) for w in list_widgets]
    # Tabs titles attribution
    for pos, title in dict_types.items():
        tab.set_title(pos, title)
    return list_widgets, tab         

def read_type(tab, list_widgets):
    """
    Reads a 'tab' widget generated by the 'gen_tab_type' function, returns the inclusion's type and aspect_ratios parameters.
    """
    type_int = tab.selected_index # Active tab
    tab_name = tab.get_title(type_int) # Active tab name
    if tab_name=='Spheres':
        # Spheres
        aspect_ratio = [1,1]
    elif tab_name=='Ellipsoids':
        # Ellipsoids
        c1 = list_widgets[type_int][0].children[1].value
        c2 = list_widgets[type_int][1].children[1].value
        aspect_ratio = [c1, c2]
    return type_int, aspect_ratio

def str_to_model(model_name):
    """
    Returns the model instance associated to the 'model_name' string. 
    """
    for Model in list_models:
        model = Model()
        if model.name.upper() == model_name.upper():
            return model
        
def incr(value, value_incr, mini, maxi):
    """
    Adds 'value_incr' to the 'value' float if it doesn't exceed mini or maxi.
    Returns a bool stating whether 'value' has been changed.
    """
    result = value
    result += value_incr
    if result > maxi:
        result = maxi
    if result < mini:
        result = mini
    changed = (result != value)
    return result, changed

---

## III- Computation homogenized behaviors of defined microstructures
This section lets the user manually generate a microstructure, then computes its homogenized behavior using available models.

In [3]:
dict_inclusions = {}
# Initialisation of the list of generated inclusions. 'dict_inclusions' format is {inclusion_name(str): inclusion(Inclusion or InclusionAndInterphase)}

# Inclusion name
w_label = widgets.Label(value='Inclusion name')
n_inclusion = 0 # Index used to automatically give each inclusion a unique name
w_name = widgets.Text(value='inclusion'+str(n_inclusion))

# Inclusion geometric type
widgets_type, tab_type = gen_tab_type() 

# Inclusion behavior
caption = widgets.Label(value='Inclusion behavior')
list_widgets, tab = gen_tab_behavior()

# Inclusion generation
button_generate_inclusion = widgets.Button(description="Generate Inclusion")
output = widgets.Output()
def generate_inclusion(b):
    """
    Called when the 'Generate inclusion' button is toggled, creates an inclusion with the chosen parameters.
    """
    global n_inclusion
    # Recovery of the chosen parameters 
    output.clear_output()
    inclusion_name = w_name.value
    if inclusion_name in list(dict_inclusions.keys()):
        with output:
            print("Name already exists")
    else :
        type_inclusion, inclusion_aspect_ratio = read_type(tab_type, widgets_type)
        behavior = read_behavior(tab, list_widgets)
        inclusion = Inclusion(type_inclusion, behavior, name=inclusion_name, aspect_ratio=inclusion_aspect_ratio)
        dict_inclusions[inclusion_name] = inclusion
        with output:
            print("Inclusion generated: ", inclusion)
        # Automatic update of the inclusion name
        n_inclusion += 1
        w_name.value = 'inclusion'+str(n_inclusion)
    
button_generate_inclusion.on_click(generate_inclusion)

# Inclusion and interphase generation
# Instance name
w_label_bis = widgets.Label(value='Instance name')
n_inclusion_bis = 0 # Index used to automatically give each inclusion a unique name
w_name_bis = widgets.Text(value='microstructure'+str(n_inclusion_bis))

# Inclusion geometric type
widgets_type_bis, tab_type_bis = gen_tab_type() 

# Inclusion behavior
caption_incl = widgets.Label(value='Inclusion behavior')
list_widgets_incl, tab_incl = gen_tab_behavior()
# Interphase behavior
caption_inter = widgets.Label(value='Interphase behavior')
list_widgets_inter, tab_inter = gen_tab_behavior()

# Instance generation
button_generate_inclusion_bis = widgets.Button(description="Generate inclusion with interphase", layout={'width': 'max-content'})
output_bis = widgets.Output()

def generate_inclusion_bis(b):
    """
    Called when the 'Generate inclusion and interphase' button is toggled, creates an InclusionAndInterphase instance with the chosen parameters.
    """
    global n_inclusion_bis
    # Recovery of the chosen parameters
    output_bis.clear_output()
    instance_name = w_name_bis.value
    if instance_name in list(dict_inclusions.keys()):
        with output_bis:
            print("Name already exists")
    else :
        type_inclusion = 0
        inclusion_aspect_ratio = [1,1]
        behavior_incl = read_behavior(tab_incl, list_widgets_incl) # behavior of the inclusion
        behavior_inter = read_behavior(tab_inter, list_widgets_inter) # behavior of the interphase
        inclusion = Inclusion(type_inclusion, behavior_incl, name=instance_name+'_inclusion', aspect_ratio=inclusion_aspect_ratio)
        interphase = Inclusion(type_inclusion, behavior_inter, name=instance_name+'_interphase', aspect_ratio=inclusion_aspect_ratio)
        instance = InclusionAndInterphase(inclusion, interphase, name=instance_name)
        dict_inclusions[instance_name] = instance
        with output_bis:
            print("Inclusion generated: ", instance)
        # Automatic update of the instance name
        n_inclusion_bis += 1
        w_name_bis.value = 'microstructure'+str(n_inclusion_bis)
    
button_generate_inclusion_bis.on_click(generate_inclusion_bis)

# Microstructure generation
microstructure = None # Initialisation

# Button linked functions
def add_inclusion_to_structure(b):
    """
    Called when the 'Add inclusion' button is toggled.
    Creates a widget linked to the volume fraction of the selected inclusion and adds it to the 'widgets_f' dict.
    Also creates a 'Remove inclusion' button and adds it to the 'buttons' dict.
    Eventually displays the generated widgets.
    """
    out2.clear_output()
    try:
        inclusion = dict_inclusions[w_inclusions.value]
    except KeyError:
        return None
    if inclusion in list(widgets_f.keys()):
        with out2:
            print("Already added")
    else:
        w_name = widgets.Label(inclusion.name)
        w_b = widgets.Button(description="Remove inclusion")
        w_b.on_click(remove_inclusion)
        buttons_suppress[w_b] = inclusion
        # Inclusion and interphase
        if type(inclusion)==InclusionAndInterphase:
            w_f_incl = widgets.FloatSlider(min=0.01, max=0.99, step=0.01, description='f inclusion')
            w_f_inter = widgets.FloatSlider(min=0.01, max=0.99, step=0.01, description='f interphase')
            widgets_f[inclusion] = (w_name, w_f_incl, w_f_inter)
            with out1:
                display(w_name, widgets.HBox([w_f_incl, w_f_inter, w_b]))
        # Simple inclusion
        else:
            w_f = widgets.FloatSlider(min=0.01, max=0.99, step=0.01, description='f')
            widgets_f[inclusion] = (w_name, w_f)
            with out1:
                display(w_name, widgets.HBox([w_f, w_b]))

def add_inclusion_to_list(b):
    """
    Called when an inclusion or inclusion and interphase is generated.
    Updates the inclusion selection widgets.
    """
    w_inclusions.options = list(dict_inclusions.keys())
    w_inclusions_info.options = list(dict_inclusions.keys())
    
    
def remove_inclusion(b):
    """
    Called when an inclusion is removed from the structure.
    Recovers the selected inclusion, closes its widgets and removes it from the 'widgets_f' dict.
    """
    out2.clear_output()
    inclusion = buttons_suppress[b]
    # Closing widgets
    b.close()
    for w in widgets_f[inclusion]:
        w.close()
    del widgets_f[inclusion]
    del buttons_suppress[b]

def generate_microstructure(b):
    """
    Generates a microstructure with the parameters set by the user.
    Displays an error message if the chosen volume fractions are not consistent.
    Eventually displays a description of the generated microstructure.
    """
    global microstructure
    matrix_behavior = read_behavior(tab_m, widgets_m) # Reading the 'Matrix behavior' widgets
    dict_inclusions = {}
    # Reading the chosen volume fractions
    for inclusion, widgets in widgets_f.items():
        # Inclusions with an interphase
        if type(inclusion)==InclusionAndInterphase:
            w_name, w_f_incl, w_f_inter = widgets
            f = [w_f_incl.value, w_f_inter.value]
        else:
            w_name, w_f = widgets
            f = w_f.value
        dict_inclusions[inclusion] = f
    # Microstructure generation
    out3.clear_output()
    try:
        microstructure = Microstructure(matrix_behavior, dict_inclusions)
        with out3:
            print("Microstructure generated\n" + str(microstructure))
            # Microstructure 3D representation
            microstructure.draw()
    except NameError:
        microstructure = None
        with out3:
            print("Inconsistent choice of volume fractions")

### Displaying inclusion widgets
# Simple inclusion
display(Markdown("## Inclusion generation"))
w_inclusion = widgets.VBox([w_label, w_name, widgets.Label(value='Inclusion type'), tab_type, caption, tab, button_generate_inclusion, output],
                          layout=layout)
display(w_inclusion)
# Inclusion with an interphase
display(Markdown("## Inclusion and interphase generation"))
w_inclusion_bis = widgets.VBox([w_label_bis,
                                w_name_bis,
                                widgets.Label(value='Inclusion type: Spheres'),
#                                tab_type_bis,
                                caption_incl,
                                tab_incl,
                                caption_inter,
                                tab_inter,
                                button_generate_inclusion_bis,
                                output_bis],
                          layout=layout)
display(w_inclusion_bis)

### Inclusion info and deletion
w_label = widgets.Label(value="Displays information on the generated inclusions")
w_inclusions_info = widgets.Dropdown(options=list(dict_inclusions.keys()), layout={'width': 'max-content'})
w_delete = widgets.Button(description="Delete inclusion", layout={'width': 'max-content'})
out_inclusions_info = widgets.Output() # Displays a description of the selected inclusion

def display_info(change):
    """
    Called when a different inclusion is selected on the 'w_inclusions_info' widget.
    Recovers the selected inclusion and displays its description.
    """
    out_inclusions_info.clear_output()
    try:
        inclusion = dict_inclusions[w_inclusions_info.value]
    except KeyError:
        inclusion = None
    with out_inclusions_info:
        print(inclusion)
        
w_inclusions_info.observe(display_info, names='value')
# Inclusions deletion
def delete_inclusion(b):
    """
    Called when an inclusion is deleted.
    Recovers the selected inclusion and deletes it from the generated inclusions dict.
    """
    inclusion_name = w_inclusions_info.value
    try:
        del dict_inclusions[inclusion_name]
    except KeyError:
        None
    w_inclusions.options = list(dict_inclusions.keys())
    w_inclusions_info.options = list(dict_inclusions.keys())

w_delete.on_click(delete_inclusion)
# Displaying inclusion info widgets
display(Markdown("## Inclusions info"))
display(widgets.VBox([w_label, widgets.HBox([w_inclusions_info, w_delete]), out_inclusions_info], layout=layout))

# Matrix behavior
caption = widgets.Label(value='Matrix behavior')
widgets_m, tab_m = gen_tab_behavior(anisotropic=False)

# Add inclusion to structure
w_inclusions = widgets.Dropdown(options=list(dict_inclusions.keys()), layout={'width': 'max-content'})
button_add_inclusion = widgets.Button(description="Add inclusion")
out1 = widgets.Output()
out2 = widgets.Output()
widgets_f = {} # Dict of the added inclusions and their widgets ('name','volume fraction')
buttons_suppress = {} # Dict of the 'Remove inclusion' buttons and their inclusions

button_add_inclusion.on_click(add_inclusion_to_structure)
button_generate_inclusion.on_click(add_inclusion_to_list)
button_generate_inclusion_bis.on_click(add_inclusion_to_list)

# Microstructure generation
b_generate_structure = widgets.Button(description='Generate microstructure', layout={'width': 'max-content'})
# TODO : widget 'valid' qui indique en temps réel si les fractions volumiques choisies sont cohérentes
out3 = widgets.Output()
b_generate_structure.on_click(generate_microstructure)

# Displays the microstructure widgets
display(Markdown("## Microstructure generation"))
w_micro = widgets.VBox([caption, tab_m, widgets.HBox([w_inclusions, button_add_inclusion, out2]), out1, b_generate_structure, out3],
                      layout=layout)
display(w_micro)

# Model selection
def test_models(b=None):
    """
    Called when a microstructure is generated.
    Tests the implemented models on the generated microstructure and creates a 'valid_models' list of compatible models.
    """
    valid_models = []
    if microstructure == None:
        # Checks whether the microstructure has been generated
        return None
    for Model in list_models:
        model = Model()
        valid = model.check_hypothesis(microstructure)
        if valid:
            # The microstructure fits the model hypothesis
            valid_models.append((model.name, model))
    # Updates the model selection widget
    select_model.options = valid_models

valid_models = [] # List of the microstructure-compatible models: [(model_name, Model)]
select_model = widgets.Dropdown()
test_models()
label = widgets.Label(value="Select a model. Only compatible models will be displayed.")
b_compute = widgets.Button(description='Compute behavior')
output_behavior = widgets.Output()

def compute_model(b):
    """
    Called when the 'Compute behavior' button is toggled.
    Recovers the selected model, computes the microstructure homogenized behavior and displays it.
    """
    output_behavior.clear_output()
    with output_behavior:
        print("Computing ...")
    model = select_model.value   
    homogenized_behavior = model.compute_h_behavior(microstructure)    
    output_behavior.clear_output()
    with output_behavior:
        print("Homogenized behavior - {} model".format(model.name))
        print(display_behavior(homogenized_behavior))
        print("When Poisson's ratio is close to 0.5, the bulk modulus K value may be erroneous")
        print(" ")
        dict_inclusions = microstructure.dict_inclusions
        instances = list(dict_inclusions.keys())
        for instance in instances:
            if (type(instance)!=InclusionAndInterphase):
                if microstructure.check_Hashin_hypothesis(): 
                    print("Hashin-Shtrikman bounds")
                    print(microstructure.Hashin_bounds())

b_generate_structure.on_click(test_models)
b_compute.on_click(compute_model)

# Displays the model widgets
display(Markdown("## Available models"))

w_model = widgets.VBox([label, widgets.HBox([select_model, b_compute]), output_behavior],
                      layout=layout)
display(w_model)

## Inclusion generation

## Inclusion and interphase generation

## Inclusions info

## Microstructure generation

## Available models

### Model comparison

Compare different models with respect to the volume fraction of filler for the matrix behavior and one of the inclusions of the last microstructure generated. 

In [4]:
import warnings
import matplotlib.cbook
warnings.filterwarnings("ignore",category=matplotlib.cbook.mplDeprecation) # Disable the matplotlib warning message
matplotlib.rcParams.update({'font.size': 6}) # Sets the font size in matplotlib graphs

# Define the max percentage for plotting
w_fmax = widgets.FloatSlider(value=0.5,min=0.01, max=0.99, step=0.01, description='fmax')


def draw_all_data(subplots):
    """
    Draws a graph with the data contained in the 'subplots' database.
    The database format is defined below.
    """
    global fig
    
    fmaxi = w_fmax.value
    out_graph.clear_output()
    # Computes the number of subplots
    parameters = list(subplots.keys())
    n_parameters = len(parameters)
    n_lines = n_parameters//2
    if n_parameters%2 == 0:
        None
    else:
        n_lines += 1
    with out_graph:
        fig, axs = plt.subplots(n_lines, 2, figsize=(8 ,n_lines*4))
        for index, parameter in enumerate(parameters):
            plt.subplot(n_lines, 2, index+1)
            list_data = subplots[parameter]
            for data in list_data:
                x, y, label = data
                if label.endswith('.txt') or label.endswith('.csv'):
                    label = label[:-4] # Deletes the extension
                if len(x)>15:
                    # Continuous representation of results
                    plt.plot(x, y, label=label)
                else:
                    # Discrete representation of results
                    plt.plot(x, y, '.', label=label)
            plt.xlabel("volume fraction")
            plt.ylabel(parameter)
            plt.title("Model comparison - "+parameter)
            if parameter=='nu':
                plt.title("Model comparison - $\\nu$")
            plt.xlim(0,fmaxi)
            plt.legend()
        plt.show()


out_graph = widgets.Output()
subplot_data = {} # Format: {parameter: subplot_data}, with subplot_data = [[f_list, values, label]] and as many lists as models to plot
plotted = [] # Already added models and text files
fig = None

# Selection of the volume fraction to play with
display(widgets.Label(value="Select an inclusion and its maximum volume fraction then click on 'Start comparing'. The results of the compared models will be plotted against the volume fraction of this inclusion"))
w_inclusion = widgets.Dropdown(layout={'width': 'max-content'})
w_setgraph = widgets.Button(description='Start comparing / Reset graph', layout={'width': 'max-content'})
display(widgets.HBox([w_inclusion, w_setgraph, w_fmax]))

def update_inclusions_list(b):
    """
    Called when a microstructure is generated.
    Updates the list of the microstructure inclusions.
    """
    global subplot_data, plotted
    inclusions = microstructure.dict_inclusions
    # Inclusions and interpahses separation
    options = {}
    for instance in inclusions:
        if type(instance)==Inclusion:
            options[instance.name] = instance
        else:
            options[instance.inclusion.name] = instance.inclusion
            options[instance.interphase.name] = instance.interphase
    w_inclusion.options = options
    out_graph.clear_output()
    subplot_data = {}
    plotted = []

if microstructure!= None:
    # Avoids the error encountered when the microstructure doesn't exist yet
    update_inclusions_list(None)
b_generate_structure.on_click(update_inclusions_list)

# Computation of a consistent list of volume fractions 'f_list'
f_list = [] 
inclusion = None # Selected inclusion

def compute_f_list(b):
    """
    Called when the 'Start comparing' button is toggled.
    Recovers the selected inclusion/interphase and computes a volume fraction list 'f_list' compatible with the rest of the inclusions.
    """
    global f_list, inclusion, subplot_data, plotted
    inclusion = w_inclusion.value
    fmaxi = w_fmax.value
    # npoints matches the number of values calculated for the differential and the self-consistent models
    # in classes_v*.py 
    npoints = max(2,int(100*fmaxi)) 
    
    # Computation of f_max
    f_max = fmaxi
    f_min=0.0
    dict_inclusions = microstructure.dict_inclusions
    for other_inclusion, f in list(dict_inclusions.items()):
        # Simple inclusions
        if type(other_inclusion)==Inclusion and other_inclusion!=inclusion:
            f_max -= f
        # Inclusions with interphase
        elif type(other_inclusion)==InclusionAndInterphase:
            f_min = 0.01
            if other_inclusion.inclusion!=inclusion:
                if (f_max+f[0] >1.0):
                    with out_progress:
                        clear_output(wait=True)
                        print('Error fmax too large')
                        return None
            if other_inclusion.interphase!=inclusion:
                if (f_max+f[1] >1.0):
                    with out_progress:
                        clear_output(wait=True)
                        print('Error fmax too large')
                        return None
    f_list = np.linspace(f_min, f_max, npoints)
    with out_progress:
        clear_output(wait=True)
        x = widgets.IntProgress(value=0,min=0,max=100,description='Progress')
        display(x)
    out_graph.clear_output()
    subplot_data = {}
    plotted = []

w_setgraph.on_click(compute_f_list)

# Adding models to graph
display(widgets.Label(value="Select a model to plot and click on 'Add model'"))
w_addmodel = widgets.Button(description="Add model")

# Follow the progress
out_progress = widgets.Output(layout={'width': 'max-content', 'border': '1px solid #FF625BF5'})
out_progress.clear_output(wait=True)
#with out_progress:
#    x = widgets.IntProgress(value=0,min=0,max=100,description='Progress')
#    display(x)
display(widgets.HBox([select_model, w_addmodel, ]),out_progress)

def plot_model(b):
    """
    Called when a model is added.
    Checks whether the model has not been added yet, adds it if it hasn't been.
    Updates the plot.
    """
    global subplot_data, plotted, f_list, inclusion, fig
    
    fmaxi = w_fmax.value
    # npoints matches the number of values calculated for the differential and the self-consistent models
    # in classes_v*.py 
    npoints = max(2,int(100*fmaxi))
    
    model = select_model.value
    if inclusion == None:
        return None
    if model not in plotted:
        ### Saving the initial volume fraction value
        # Recovery of the associated instance when the inclusion is part of an InterphaseAndInclusion instance
        if inclusion.inc_and_int==None:
            effective_inclusion = inclusion
        else:
            effective_inclusion = inclusion.inc_and_int[0]
        
        f_old = microstructure.dict_inclusions[effective_inclusion] 
        plotted.append(model)
        # Computation of the list of the behaviors for the different values of f
        list_behaviors = {} # Format: {parameter: [values against f]}
        if (inclusion.type_inclusion == 1 or 'K' not in list(inclusion.behavior.keys())) and (model.name == 'Differential scheme' or model.name == 'Self-consistent'):
            with out_progress:
                    clear_output(wait=True)
                    print('Computing...')
        # Compute the model only once for fmaxi
            microstructure.change_fi(inclusion,fmaxi)
            h_behavior = model.compute_h_behavior(microstructure)
            h_behavior = {}
            if (model.name =='Differential scheme'):
                file_to_read = "outputs/model_comparison/last_differential.txt"
            else:
                file_to_read = "outputs/model_comparison/last_selfconsistent.txt"
            # read the results
            with open(file_to_read, 'r') as filemod:
                lines = filemod.readlines()
                for n_lines,line in enumerate(lines[0:]):
                    line1 = line.strip().split(',')
                    h_behavior['K']=float(line1[1])
                    h_behavior['G']=float(line1[2])
                    h_behavior['E']=float(line1[3])
                    h_behavior['nu']=float(line1[4])
                    for parameter, value in h_behavior.items():
                        if parameter not in list_behaviors.keys():
                            # The parameter is encountered for the first time
                            list_behaviors[parameter] = []
                        list_behaviors[parameter].append(value)
            with out_progress:
                    clear_output(wait=True)
                    print('Done :)')
                    
        else:    
            for f in f_list:
                with out_progress:
                    clear_output(wait=True)
                    x = widgets.IntProgress(value=int(f*100),min=0,max=npoints,description='Progress')
                    display(x)
                # Simple inclusion
                if inclusion.inc_and_int == None:
                    microstructure.change_fi(inclusion, f)
                # Inclusion with an interphase
                else:
                    instance, pos = inclusion.inc_and_int # pos equals to 0 for an inclusion and 1 for the interphase
                    # Computes the new volume fraction list for the InclusionAndInterphase instance
                    new_f = microstructure.dict_inclusions[instance]
                    new_f[pos] = f
                    # Incrementation of the volume fraction
                    microstructure.change_fi(instance, new_f)
            
                h_behavior = model.compute_h_behavior(microstructure)
                if 'K' not in list(h_behavior.keys()):
                    h_behavior = Isotropic_behavior(h_behavior)
                
                for parameter, value in h_behavior.items():
                    if parameter not in list_behaviors.keys():
                        # The parameter is encountered for the first time
                        list_behaviors[parameter] = []
                    list_behaviors[parameter].append(value)
                    
        # Update of the 'subplot_data' database
        for parameter, values in list_behaviors.items():
            data = [f_list, values, model.name]
            # Creation of the parameter entry when it is ploted for the first time
            if parameter not in subplot_data.keys():
                subplot_data[parameter] = []
            subplot_data[parameter].append(data)
        # Reversing the microstructure changes
        microstructure.change_fi(effective_inclusion, f_old)
        # Plotting data
        draw_all_data(subplot_data)
                    
w_addmodel.on_click(plot_model)

# Adding data from text files
display(widgets.Label(value="Plot data from a text file. Input files are in the 'inputs/model_comparison' folder. See 'example.txt' for the format"))
list_files = listdir('inputs/model_comparison/')
w_file = widgets.Dropdown(options=[file for file in list_files if file.endswith('.txt')])
w_add_data = widgets.Button(description="Add data")
display(widgets.HBox([w_file, w_add_data]))

def plot_data(b):
    """
    Called when the 'Add data' button is toggled.
    Checks wheteher the selected file has been added to the graph, adds it if it hasn't been.
    Updated the graph.
    """
    global subplot_data, plotted, fig
    file_name = w_file.value
    if file_name not in plotted:
        plotted.append(file_name)
        df = pd.read_csv('inputs/model_comparison/'+file_name)
        # Recovery of the volume fraction list
        try:
            f_values = df['f']
        except KeyError:
            with out_graph:
                print("Wrong format")
        # Parameters recovery
        parameters = df.keys()
        for parameter in parameters:
            if parameter == 'f':
                continue
            # Creation of the parameter entry
            if parameter not in subplot_data.keys():
                subplot_data[parameter] = []
            values = list(df[parameter])
            subplot_data[parameter].append([f_values, values, file_name]) 
        # Graph plotting
        draw_all_data(subplot_data)
    
w_add_data.on_click(plot_data)

# Hashin bounds
w_addbounds = widgets.Button(description="Add HS bounds")
display(widgets.Label(value="Add Hashin-Shtrikman bounds to figures"))
display(w_addbounds)

def plot_bounds(b):
    """
    Called when the ashin bounds are added.
    Adds the Hashin bounds to the graph if they have not been added yet.
    Updates the graph.
    """
    global subplot_data, plotted, f_list, inclusion
    if "bounds" not in plotted and microstructure.Hashin_bounds()!=None:
        plotted.append("bounds")
        f_old = microstructure.dict_inclusions[inclusion] # Savec the initial volume fraction value
        # Computes the list of behaviors for the different values of f
        list_behaviors = {} # Format: {parameter: [values against f]}
        for f in f_list:
            microstructure.change_fi(inclusion, f)
            h_bounds = microstructure.Hashin_bounds()
            for parameter, value in h_bounds.items():
                if parameter not in list_behaviors.keys():
                    # First encounter of the parameter
                    list_behaviors[parameter] = []
                list_behaviors[parameter].append(value)
        # Update of the 'subplot_data' database
        for parameter, values in list_behaviors.items():
            parameter_key = parameter[:-3] # Deletes the 'inf' and 'sup' strings
            parameter_suffix = parameter[-3:] # 'inf' or 'sup'
            if parameter_suffix=='inf':
                suffix = 'lower'
            else:
                suffix = 'upper'
            label = "HS " + suffix + " bound " + parameter_key
            data = [f_list, values, label]
            # Creates the entry parameter if it is plotted for the first times
            if parameter_key not in subplot_data.keys():
                subplot_data[parameter_key] = []
            subplot_data[parameter_key].append(data)
        # Reverse the microstructure changes
        microstructure.change_fi(inclusion, f_old)
        # Drawing the data
        draw_all_data(subplot_data)

w_addbounds.on_click(plot_bounds)

# Saving the figures
n_fig = 0 # Figure index
w_save_image = widgets.Button(description="Save figures")
w_filename = widgets.Text(value='fig0.pdf')
display(widgets.Label(value="Enter a valid file name with an extension (ex: .pdf, .png, .jpg) and click on save to save the figures. The output file will be saved in the 'outputs/model_comparison' folder"))
display(widgets.HBox([w_filename, w_save_image]))

def save_image(b):
    """
    Called when a figure is saved. Saves the figure in the 'outputs/model_comparison' fodler with the selected name.
    Automatically changes the name of the next figure to be saved.
    """
    global fig, n_fig
    filename = w_filename.value
    fig.savefig("outputs/model_comparison/"+filename)
    # Automatic update of the name
    n_fig += 1
    w_filename.value = 'fig{}.pdf'.format(n_fig)
    
w_save_image.on_click(save_image)

# Saving the data
n_data = 0 # Index of the data file
w_save_data = widgets.Button(description="Save data")
w_data_filename = widgets.Text(value='data0.csv')
display(widgets.Label(value="Enter a valid file name with an extension (ex: .txt, .csv) and click on save to save the figures data. The ouput file will be saved in the 'outputs/model_comparison' folder"))
display(widgets.HBox([w_data_filename, w_save_data]))

def save_data(b):
    """
    Called when data is saved. Saves the data (txt) in the 'ouputs.model_comparison' folder with the selected name.
    Automatically changes the name of the next data file to be saved.
    """ 
    global subplot_data, n_data
    filename = w_data_filename.value
    # clear out file
    # Automatic update of the name
    n_data += 1
    w_data_filename.value = 'data{}.csv'.format(n_data)
    # Creation of a dataframe containing the data
    data = {}
    # Recovery of the models data 
    for parameter, parameter_data in subplot_data.items():
        for model_data in parameter_data:
            f_list, model_values, label = model_data
            if label.endswith('.txt') == False and label.endswith('.csv') == False:
                # The data refers to a model and not to an input file
                data["volume fraction f"] = f_list
                data[parameter + " - " + label] = model_values
    df = pd.DataFrame(data)
    df.to_csv('outputs/model_comparison/'+filename, header=True, index=False, sep=',', mode='a')

w_save_data.on_click(save_data)
    
display(out_graph)

Label(value="Select an inclusion and its maximum volume fraction then click on 'Start comparing'. The results …

Label(value="Select a model to plot and click on 'Add model'")

Output(layout=Layout(border='1px solid #FF625BF5', width='max-content'))

Label(value="Plot data from a text file. Input files are in the 'inputs/model_comparison' folder. See 'example…

Label(value='Add Hashin-Shtrikman bounds to figures')

Button(description='Add HS bounds', style=ButtonStyle())

Label(value="Enter a valid file name with an extension (ex: .pdf, .png, .jpg) and click on save to save the fi…

Label(value="Enter a valid file name with an extension (ex: .txt, .csv) and click on save to save the figures …

Output()

---

## IV- Automatic computation from a .txt file
This section has been created to compute some homogenized behavior quickly without going through the previous sections. This is especially interesting when one want to compute a lot of different microstructures.
An example of input file can be found in the folder inputs/automatic_computation.
Giving the model, the matrix behavior, the geometry of the inclusions, volume fraction, the behavior of the inclusion.

In [ ]:
list_inputs = [] # List of the compatible files in the input folder
folder = 'inputs/automatic_computation/' # Input folder

# Recovery of the files
def compatible_file(file_name, folder):
    """
    Checks whether a given file 'file_name' in the 'folder' folder has the wanted format, meaning:
    - the file is a .txt file
    - its first line is '*homogenization'
    Returns a bool.
    """
    result = True # Initialisation
    # File name test
    if len(file_name)<5 or file_name[-4:]!='.txt':
        result = False
    else:
        # First line reading
        with open(folder+file_name, 'r') as file:
            line = file.readline()
            if line.strip() != '*homogenization':
                result = False
    return result

def check_files(folder=folder):
    """
    Called when the 'refresh list' button is toggled.
    Updates the 'list_inputs' list of compatible files in the input folder.
    """
    global list_inputs
    list_inputs_raw = listdir(folder)
    list_inputs = [] # Resets the list
    for file_name in list_inputs_raw:
        if compatible_file(file_name, folder):
            list_inputs.append(file_name)

def read_file(b):
    """
    Called when the 'Generate output file' button is toggled.
    Reads the name of the selected file, opens and reads the file.
    Displays an error message if an error is detected.
    Else, computes the homogenized behavior of each microstructure and creates an output file in the 'outputs/automatic_computation' folder.
    TODO: adapter au calcul avec des modèles différents donnant des paramètres de comportement différents
    """
    folder_in = 'inputs/automatic_computation/'
    folder_out = 'outputs/automatic_computation/'
    file_name = w_file.value
    file_name_out = file_name[:-4] + '_out.csv' # Adds the '.csv' extension
    out_file.clear_output()
    read_matrix = False # Defines whether the current line refers to the definition of a new microstructure or of an inclusion
    read_model = True # Defines if the active line refers to a model selection
    dict_inclusions = {}
    n = 0
    n_inclusion = 0 # Index used to automatically give each inclusion a unique name
    inclusion_name = "inclusion"+str(n_inclusion)
    # Output file initialisation
    with open(folder_out+file_name_out, 'w') as file_out:
        file_out.write("K,G,E,nu\n")
    # Reading the input file
    with open(folder_in+file_name, 'r') as file:
        lines = file.readlines()
        nbcalc=0
        for n_line, line in enumerate(lines[1:]):
            if line.strip() == '*':
                nbcalc+= 1
        for n_line, line in enumerate(lines[1:]):
            try:
                if read_model:
                    # Model definition
                    model_name = line.strip()
                    model = str_to_model(model_name)
                    # Going to next line
                    read_model = False
                    read_matrix = True
                elif read_matrix:
                    # Matrix behavior reading
                    matrix_behavior = {}
                    line1 = line.strip().split(',')
                    for parameter in line1:
                        parameter = parameter.split(':')
                        matrix_behavior[parameter[0]] = float(parameter[1])
                    # Going to next line
                    read_matrix = False
                elif line.strip() == '*':
                    # Computation of the previous microstructure's homogenized behavior
                    microstructure = Microstructure(matrix_behavior, dict_inclusions)
                    behavior_h = model.compute_h_behavior(microstructure)
                    if 'K' not in list(behavior_h.keys()):
                        behavior_h = Isotropic_behavior(behavior_h)
                    values=[behavior_h['K'],behavior_h['G'],behavior_h['E'],behavior_h['nu']]
                    values = [str(e) for e in values]
                        
                    # Writing the behavior in the output file
                    with open(folder_out+file_name_out, 'a') as file_out: 
                        file_out.write(','.join(values)+'\n')
                    # Going to next line
                    read_model = True
                    n_inclusion = 0
                    dict_inclusions = {}
                    n += 1
                else:
                    # Reading an inclusion
                    line1 = line.strip().split(',')
                    type_inclusion = 0
                    aspect_ratio_inclusion = [1.0,1.0]
                    if (float(line1[0])>0.0001):
                        type_inclusion = 1
                        aspect_ratio_inclusion = [float(line1[0]),float(line1[1])]
                    f = line1[2] # volume fraction
                    # Beahavior of the inclusion
                    inclusion_behavior = {}
                    # Test if model 4-phase model then read inclusion + interphase
                    modelstring = str(model)
                    substring_4phases = "4"
                    if(modelstring.count(substring_4phases)>0): 
                        interphase_behavior = {}
                        f=[float(line1[1]),float(line1[4])]
                        for parameter in line1[2:4]:
                            parameter = parameter.strip().split(':')
                            inclusion_behavior[parameter[0]] = float(parameter[1])
                        for parameter in line1[5:7]:
                            parameter = parameter.strip().split(':')
                            interphase_behavior[parameter[0]] = float(parameter[1])   
                        incl = Inclusion(type_inclusion, inclusion_behavior, aspect_ratio=aspect_ratio_inclusion)
                        interphase = Inclusion(type_inclusion, interphase_behavior, aspect_ratio=aspect_ratio_inclusion)
                        inclusion = InclusionAndInterphase(incl, interphase)
                        dict_inclusions[inclusion] = f
                    else: # For an inclusion
                        # Test if the inclusion is isotropic
                        maxstring=str(line1)
                        substring_iso= ":"
                        if(maxstring.count(substring_iso)>0):
                            for parameter in line1[3:5]:
                                parameter = parameter.strip().split(':')
                                inclusion_behavior[parameter[0]] = float(parameter[1])
                        else: # the filler is anisotropic
                            parameter = str(line1[3]) # Name of the parameter compliance or stiffness
                            w_parameter = str(line1[4]) # Name of the .txt file with the matrix behavior
                            file_name_behave = "inputs/automatic_computation/" + w_parameter
                        # Anisotropic behavior file reading
                            value_incl = []
                            with open(file_name_behave) as csvfile:
                                reader = csv.reader(csvfile, quoting=csv.QUOTE_NONNUMERIC) # change contents to floats
                                for row in reader: # each row is a list
                                    value_incl.append(row)
                        # Generation of the 'behavior' dict
                            inclusion_behavior = {parameter: np.array(value_incl)} 
                    # Inclusion generation
                        inclusion_name = Inclusion(int(type_inclusion), inclusion_behavior, name=inclusion_name, aspect_ratio = aspect_ratio_inclusion)
                        dict_inclusions[inclusion_name] = float(f) 
                        n_inclusion += 1
                        inclusion_name = 'inclusion'+str(n_inclusion)
            except:
                with out_file:
                    print("Error on line {}: {} ".format(n_line+1, line))
                    return None
            with out_file:
                clear_output(wait=True)
                x = widgets.IntProgress(value=n,min=0,max=nbcalc,description='Progress')
                display(x)
    with out_file:
        print("Output file generated in the 'outputs/automatic_computation' folder ")

def refresh(b):
    """
    Called when the 'Refresh input files' button is toggled.
    Updates the compatible input files list and widget.
    """
    check_files()
    w_file.options = list_inputs

b_refresh = widgets.Button(description='Refresh input files list')
display(b_refresh)
w_label = widgets.Label(value='Choose an input file :')
w_file = widgets.Dropdown(options=list_inputs)
refresh(None) # Update of the available input files
b_compute = widgets.Button(description='Generate output file')
display(widgets.HBox([w_label, w_file, b_compute]))
out_file = widgets.Output(layout={'width': 'max-content', 'border': '1px solid #FF625BF5'})
display(out_file)
out_file.clear_output(wait=True)
with out_file:
    x = widgets.IntProgress(value=0,min=0,max=10,description='Progress')
    display(x)
    print("Press 'Generate output file' to compute ")
    
b_refresh.on_click(refresh)
b_compute.on_click(read_file)

message = """If your file does not appear in the menu:
- Check that your file is in the 'inputs' folder
- Check that your file is a '.txt' file with first line as '*homogenization'
- Press the 'Refresh input files list' button"""

print(message)

---

## V- Model description

For educational purpose, this part gives a short survey on the models equations and references.

To add a model here, simply write its description in a Markdown file (.md) in the 'model_descriptions' folder with first line '# Model name'


In [ ]:
from IPython.display import Latex, Markdown
from os import listdir

# Recovery of the model descriptions files  
folder = 'model_descriptions/'
folder_files = listdir(folder)
descriptions = [] # List of model descriptions files, format: ['model_name', 'file_path']
for file_name in folder_files:
    if file_name.endswith('.md'):
        path = folder + file_name
        with open(path, 'r') as opened_file:
            title = opened_file.readline()
        model_name = title[2:].strip() # Deletes the '#' at the beginning
        descriptions.append((model_name, path))

# Displays the description
w_description = widgets.Dropdown(options=descriptions)
display(w_description)
out_description = widgets.Output(layout=layout)
display(out_description)

def display_description(change):
    """
    Called when a model is selected.
    Recovers the model and displays its description.
    """
    out_description.clear_output()
    file_name = w_description.value
    with open(file_name, 'r') as file:
        description = file.read()
        with out_description:
            display(Markdown(description))
            
display_description(None)
w_description.observe(display_description, names='value')


---

## VI- Estimates of parameters by an inverse method for isotropic composites

This section enables the computation of a constitutive phase isototropic behavior or the inclusion volume fraction for an isotropic composite material. 
The user enters a target behavior he would like to reach for the composite material, then builds a microstructure by choosing the inclusions, their type, their behavior and volume fractions, along with the behavior of the matrix. At each step, the user can set a parameter as unknown, which is the parameter that will be optimized.


In [ ]:
def gen_tab_behavior(inverse=False):
    """
    Generate a 'tab' widget defining the behavior parameters.
    Each generated tab refers to a behavior from the 'dict_behaviors' dictionnary defined in 'classes.py'.
    The 'isotropic' parameter is 'True' if one only wants to display isotropic behaviors.
    The function returns:
    - A list of list of widgets 'list_widgets' with as many lists as tabs. Each list contains the widgets for the parameters associated with the behavior of the corresponding tab.
    - The tab widget to be displayed.
    """
    behaviors_str = list(dict_behaviors.keys()) # Lists the names of the behaviors implemented in classes.py
    list_widgets = [] # List of lists, each list refers to a tab of the final widget and contains the non-formatted widgets of this tab
    tab_titles = [] # Tabs names, each tab refers to a behavior type ('Isotropic K and G', 'Anisotropic', etc..) 
    # Generation of the widgets for each tab
    for behavior_str in behaviors_str:
        if 'Anisotropic' in behavior_str:
            continue
        widgets_onglet = []
        parameters = dict_behaviors[behavior_str] # Behavior-associated parameters (example : ['K', 'G'] for Isotropic)
        for parameter in parameters:
            w = widgets.BoundedFloatText(value=200, min=0, max=10**12, step=1) # 'parameter' associated widget
            if parameter == 'nu':
                w.max = 0.5
                w.value = 0.3
                w.step = 0.1
            w_label = widgets.Label(value=parameters_name[parameter])
            if parameter != 'nu' and inverse:
                w_checkbox = widgets.Checkbox(value=False, description="Set as unknown")
                widget_onglet = widgets.HBox([w_label, w, w_checkbox])
            else:
                widget_onglet = widgets.HBox([w_label, w])
            widgets_onglet.append(widget_onglet)
        list_widgets.append(widgets_onglet)
        tab_titles.append(behavior_str)
    # Tab creation
    tab = widgets.Tab()
    tab.children = [widgets.VBox(w) for w in list_widgets]
    for pos, title in enumerate(tab_titles):
        tab.set_title(pos, title)
    return list_widgets, tab

def read_behavior(tab, list_widgets, inverse=False):
    """
    Reads tab widgets created by the 'gen_tab_behavior'.
    Returns a 'behavior' dictionnary and a list of unknown parameters if 'inverse' is True.
    """
    behavior_int = tab.selected_index # Index of the tab selected by the user
    widgets_parameters = list_widgets[behavior_int] # Parameter widgets of the active tab
    behavior = {}
    unknown_parameters = [] # Unknown parameters list
    for w in widgets_parameters:
        name = w.children[0].value
        value = w.children[1].value
        # Parameter true name
        name = parameters_name_bis[name]
        # Entry creation
        if inverse:
            try:
                # Avoids the error due to the absence of the 'set as unknown' checkbox (nu)
                unknown = w.children[2].value
                if unknown:
                    # The parameter is unknown
                    unknown_parameters.append(name)
            except:
                None
        behavior[name] = value
    if inverse:
        return behavior, unknown_parameters
    else:
        return behavior

### Target behavior
w_target = [] # Target behavior widgets list
target_behavior = {} # Initialisation
ddl = {} # List of the degrees of freedom of the optimization algorithm
# Format: {function: [min, max]}
# function takes three input arguments, an increment (float), a min and a max
# It increments a parameter (ex: volume fraction of an inclusion) if it stays between its bounds
widgets_target, tab_target = gen_tab_behavior()
b_target = widgets.Button(description="Generate target behavior", layout={'width': 'max-content'})
out_target = widgets.Output()


# Behavior generation
def generate_target(b):
    """
    Called when a target behavior is generated.
    """
    global target_behavior
    # Recovery of the selected behavior
    out_target.clear_output()
    target_behavior = read_behavior(tab_target, widgets_target)
    with out_target:
        print("Target behavior generated : {}".format(target_behavior))

b_target.on_click(generate_target)

# Displays the target widgets
w_target = [tab_target, b_target, out_target]
display(Markdown("## Target behavior"))
display(widgets.VBox(w_target, layout=layout))

### Inclusion generation
dict_inclusions = {}
# Initialisation of the list of created inclusions. Format: {name_inclusion (str): inclusion (Inclusion or InclusionAndInterphase)}
unknown_inclusion_parameters = {}
# Contains the unknown parameters of each inclusion, format: {inclusion: [unknown parameters]}

# Inclusion name
w_label = widgets.Label(value='Inclusion name')
n_inclusion = 0 # Index used to give each inclusion a unique name
w_name = widgets.Text(value='inclusion'+str(n_inclusion))

# Inclusion type
caption_type = widgets.Label(value='Inclusion type')
widgets_type, tab_type = gen_tab_type() # Inclusion type tab generation

# Inclusion behavior
caption = widgets.Label(value='Inclusion behavior')
list_widgets, tab = gen_tab_behavior(inverse=True)

# Inclusion generation
button_generate_inclusion = widgets.Button(description="Generate Inclusion")
output = widgets.Output()
def generate_inclusion(b):
    """
    Called when an inclusion is generated, creates an inclusion with the selected parameters.
    """
    global n_inclusion, unknown_inclusion_parameters
    # Recovery of the selected inclusions
    output.clear_output()
    inclusion_name = w_name.value
    if inclusion_name in list(dict_inclusions.keys()):
        with output:
            print("Name already exists")
    else :
        type_inclusion, inclusion_aspect_ratio = read_type(tab_type, widgets_type)
        behavior, unknown_parameters = read_behavior(tab, list_widgets, inverse=True)
        inclusion = Inclusion(type_inclusion, behavior, name=inclusion_name, aspect_ratio=inclusion_aspect_ratio)
        dict_inclusions[inclusion_name] = inclusion
        w_inclusions_info.options = list(dict_inclusions.keys())
        with output:
            print("Inclusion generated: ", inclusion)
            print("Unknown parameters: ", unknown_parameters)
        # Automatic update of the inclusion name
        n_inclusion += 1
        w_name.value = 'inclusion'+str(n_inclusion)
        # Saves the unknown parameters
        unknown_inclusion_parameters[inclusion] = unknown_parameters
    
button_generate_inclusion.on_click(generate_inclusion)

# Displays the inclusion widgets 
w_inclusions = [w_label, w_name, caption_type, tab_type, caption, tab, button_generate_inclusion, output]
display(Markdown("## Inclusion generation"))
display(widgets.VBox(w_inclusions, layout=layout))

### Inclusion and interpahse generation
# Inclusion name
w_label_bis = widgets.Label(value='Instance name')
n_inclusion_bis = 0 # Index used to give each instance a unique name
w_name_bis = widgets.Text(value='microstructure'+str(n_inclusion_bis))

# Inclusion type
widgets_type_bis, tab_type_bis = gen_tab_type() # InclusionAndInterpahse type tab generation

# Inclusion behavior
caption_incl = widgets.Label(value='Inclusion behavior')
list_widgets_incl, tab_incl = gen_tab_behavior(inverse=True)
# Interphase behavior
caption_inter = widgets.Label(value='Interphase behavior')
list_widgets_inter, tab_inter = gen_tab_behavior(inverse=True)

# Inclusion generation
button_generate_inclusion_bis = widgets.Button(description="Generate inclusion with interphase", layout={'width': 'max-content'})
output_bis = widgets.Output()

def generate_inclusion_bis(b):
    """
    Called when an inclusion with interphase is generated. Creates an InclusionAndInterphase instance.
    """
    global n_inclusion_bis
    # Recovery of the selected parameters
    output_bis.clear_output()
    instance_name = w_name_bis.value
    if instance_name in list(dict_inclusions.keys()):
        with output_bis:
            print("Name already exists")
    else :
        type_inclusion = 0 
        inclusion_aspect_ratio = [1,1]
        behavior_incl, unknown_parameters_incl = read_behavior(tab_incl, list_widgets_incl, inverse=True) # behavior of the inclusion
        behavior_inter, unknown_parameters_inter = read_behavior(tab_inter, list_widgets_inter, inverse=True) # behavior of the interphase
        inclusion = Inclusion(type_inclusion, behavior_incl, name=instance_name+'_inclusion', aspect_ratio=inclusion_aspect_ratio) # Creation of the inclusion
        interphase = Inclusion(type_inclusion, behavior_inter, name=instance_name+'_interphase', aspect_ratio=inclusion_aspect_ratio) # Creation of the interphase
        instance = InclusionAndInterphase(inclusion, interphase, name=instance_name) # Creation of the instance InclusionAndInterphase
        # Inclusions list update
        dict_inclusions[instance_name] = instance
        w_inclusions_info.options = list(dict_inclusions.keys())
        # Unknwon parameters dict update
        unknown_inclusion_parameters[inclusion] = unknown_parameters_incl
        unknown_inclusion_parameters[interphase] = unknown_parameters_inter
        with output_bis:
            print("Inclusion generated: ", instance)
            print("Unknown parameters - inclusion: ", unknown_parameters_incl)
            print("Unknown parameters - interphase: ", unknown_parameters_inter)
        # Automatic update of the instance name
        n_inclusion_bis += 1
        w_name_bis.value = 'microstructure'+str(n_inclusion_bis)
    
button_generate_inclusion_bis.on_click(generate_inclusion_bis)
display(Markdown("## Inclusion and interphase generation"))
w_inclusion_bis = widgets.VBox([w_label_bis,
                                w_name_bis,
                                widgets.Label(value='Inclusion type: Spheres'),
#                                tab_type_bis,
                                caption_incl,
                                tab_incl,
                                caption_inter,
                                tab_inter,
                                button_generate_inclusion_bis,
                                output_bis],
                          layout=layout)
display(w_inclusion_bis)

### Inclusions info and deletion
w_label = widgets.Label(value="Displays informations on the generated inclusions")
w_inclusions_info = widgets.Dropdown(options=list(dict_inclusions.keys()), layout={'width': 'max-content'})
w_delete = widgets.Button(description="Delete inclusion", layout={'width': 'max-content'})
out_inclusions_info = widgets.Output()
# Displays the selected inclusion info
def display_info(change):
    """
    Called when an inclusion is selected, displays its description.
    """
    out_inclusions_info.clear_output()
    try:
        inclusion = dict_inclusions[w_inclusions_info.value]
    except KeyError:
        inclusion = None
    with out_inclusions_info:
        print(inclusion)
        
w_inclusions_info.observe(display_info, names='value')
# Inclusion deletion
def delete_inclusion(b):
    """
    Called when an inclusion is deleted.
    Recovers the inclusion and deletes it from the generated inclusions dict.
    """
    inclusion_name = w_inclusions_info.value
    try:
        del dict_inclusions[inclusion_name]
    except KeyError:
        None
    w_inclusions.options = list(dict_inclusions.keys())
    w_inclusions_info.options = list(dict_inclusions.keys())

w_delete.on_click(delete_inclusion)
# Displays inclusions info widgets
display(Markdown("## Inclusions info"))
display(widgets.VBox([w_label, widgets.HBox([w_inclusions_info, w_delete]), out_inclusions_info], layout=layout))

### Microstructure generation
microstructure = None # Initialisation

def add_inclusion_to_structure(b):
    """
    Called when the 'Add inclusion' button is toggled.
    Creates a widget linked to the volume fraction of the selected inclusion and adds it to the 'widgets_f' dict.
    Also creates a 'Remove inclusion' button and adds it to the 'buttons' dict.
    Eventually displays the generated widgets.
    """
    out2.clear_output()
    try:
        inclusion = dict_inclusions[w_inclusions.value]
    except KeyError:
        return None
    if inclusion in list(widgets_f.keys()):
        with out2:
            print("Already added")
    else:
        w_name = widgets.Label(inclusion.name) # Added inclusion name
        w_b = widgets.Button(description="Remove inclusion")
        w_b.on_click(remove_inclusion)
        buttons_suppress[w_b] = inclusion # Button and inclusion association
        # Simple inclusions
        if type(inclusion)==Inclusion:
            w_f = widgets.FloatSlider(min=0.01, max=0.99, step=0.01, description='f') # volume fraction widget
            w_c = widgets.Checkbox(value=False, description="Set as unknown")
            widgets_f[inclusion] = (w_name, w_f, w_c) # Widgets referring to the inclusion
            with out1:
                display(widgets.HBox([w_name, w_b]), widgets.HBox([w_f, w_c]))
        # Inclusions with interphase
        else:
            w_f_incl = widgets.FloatSlider(min=0.01, max=0.99, step=0.01, description='f_inclusion')
            w_f_int = widgets.FloatSlider(min=0.01, max=0.99, step=0.01, description='f_interphase')
            w_c_incl = widgets.Checkbox(value=False, description="Set as unknown")
            w_c_int = widgets.Checkbox(value=False, description="Set as unknown")
            widgets_f[inclusion] = (w_name, w_f_incl, w_c_incl, w_f_int, w_c_int) # Widgets referring to the inclusion
            with out1:
                display(widgets.HBox([w_name, w_b]), widgets.HBox([w_f_incl, w_c_incl]), widgets.HBox([w_f_int, w_c_int]))
            
def add_inclusion_to_list(b):
    """
    Called when an inclusion or inclusion and interphase is generated.
    Updates the inclusion selection widgets.
    """
    w_inclusions.options = list(dict_inclusions.keys())
    
def remove_inclusion(b):
    """
    Called when an inclusion is removed from the structure.
    Recovers the selected inclusion, closes its widgets and removes it from the 'widgets_f' dict.
    """
    out2.clear_output()
    inclusion = buttons_suppress[b] # Recovery of the inclusion associated to the inclusion
    # Closing widgets
    for widget in widgets_f[inclusion]:
        widget.close()
    b.close()
    # Widgets list update
    del widgets_f[inclusion]
    del buttons_suppress[b]

# Dynamic generation of functions associated to degrees of freedom
def make_function_f(inclusion, index=None):
    """
    Creates a function that change an inclusion volume fraction
    """
    def function(value_incr, mini, maxi):
        """
        Increments the value of the unknown volume fraction by 'value_incr' (float)
        mini and maxi are the bounds of the parameter.
        Returns a True boolean if the modified value of the parameter is within the bounds.
        """
        if index==None:
            value = microstructure.dict_inclusions[inclusion]
            value, changed = incr(value, value_incr, mini, maxi)
            microstructure.change_fi(inclusion, value)
        else:
            value = microstructure.dict_inclusions[inclusion]
            value[index], changed = incr(value[index], value_incr, mini, maxi)
            microstructure.change_fi(inclusion, value)
        return changed
    return function

def make_function_mbehavior(parameter):
    """
    Creates a function that modifies the value of a matrix behavior parameter.
    parameter: str (example : 'K', 'G')
    """
    def function(value_incr, mini, maxi):
        value = microstructure.behavior[parameter]
        value, changed = incr(value, value_incr, mini, maxi)
        microstructure.change_parameter(parameter, value)
        return changed
    return function

def make_function_ibehavior(inclusion, parameter):
    """
    Creates a function that modifies the value of the inclusion parameter.
    parameter: str (example : 'K', 'G')
    """ 
    def function(value_incr, mini, maxi):
        value = inclusion.behavior[parameter]
        value, changed = incr(value, value_incr, mini, maxi)
        inclusion.change_parameter(parameter, value)
        return changed
    return function

def generate_microstructure(b):
    """
    Generates a microstructure with the parameters set by the user.
    Displays an error message if the chosen volume fractions are not consistent.
    Eventually displays a description of the generated microstructure.
    Updates the degrees of freedom list for the optimization algorithm.
    Met à jour la liste des degrés de liberté de l'algorithme d'optimization.
    """
    global microstructure, ddl
    ddl = {}
    matrix_behavior, unknown_parameters = read_behavior(tab_m, widgets_m, inverse=True) # Reading the 'Matrix beahvior' widgets
    dict_inclusions = {}
    # Generation of the unknown parameters functions
    for parameter in unknown_parameters:
        function = make_function_mbehavior(parameter)
        ddl[function] = [0.01, 10**6, 0.05]
    # Adding inclusions
    dict_inclusions = {}
    # Reading the selected volume fractions
    for inclusion, widgets in widgets_f.items():
        # Simple inclusions
        if type(inclusion)==Inclusion:
            w_name, w_f, w_c = widgets
            f = w_f.value # Selected volume fraction
            unknown_f = w_c.value # True if f is unknwon
            if unknown_f:
                f = 0.01 # Initialisation of the unknown value
                # Definition of the function associated to the degree of freedom
                function = make_function_f(inclusion)
                ddl[function] = [0.01, 0.99, 0.0001]
            dict_inclusions[inclusion] = f # Generation of the microstructure attribute
        # Inclusions with interphase
        else:
            w_name, w_f_inc, w_c_inc, w_f_int, w_c_int = widgets
            f_inc, f_int = w_f_inc.value, w_f_int.value
            unknown_f_inc, unknown_f_int = w_c_inc.value, w_c_int.value
            if unknown_f_inc:
                f_inc = 0.01 # Initialisation of the unknown parameter
                # Definition of the function associated to the degree of freedom
                function = make_function_f(inclusion, index=0)
                ddl[function] = [0.01, 0.99, 0.0001]
            if unknown_f_int:
                f_int = 0.01 # Initialisation of the unknown value
                # Definition of the function associated to the degree of freedom
                function = make_function_f(inclusion, index=1)
                ddl[function] = [0.01, 0.99, 0.0001]
            dict_inclusions[inclusion] = [f_inc, f_int] # Generation of the microstructure attribute
    # Generation of functions associated to unknown inclusion behavior parameters
    instances = [] # Instance list (inclusions or interphases)
    for inclusion in dict_inclusions.keys():
        if type(inclusion)==Inclusion:
            instances.append(inclusion)
        else:
            instances += [inclusion.inclusion, inclusion.interphase]
    for inclusion in instances:
        try:
            unknown_parameters = unknown_inclusion_parameters[inclusion]
            for parameter in unknown_parameters:
                function = make_function_ibehavior(inclusion, parameter)
                ddl[function] = [0.01, 10**6, 0.05]
        except KeyError:
            # All the inclusion parameters are known
            None
    # Microstructure generation
    out3.clear_output()
    try:
        microstructure = Microstructure(matrix_behavior, dict_inclusions)
        with out3:
            print("Microstructure generated\n" + str(microstructure))
            #print("Unknown parameters: ", unknown_parameters)
    except NameError:
        microstructure = None
        with out3:
            print("Inconsistent choice of volume fractions")

# Matrix behavior
caption = widgets.Label(value='Matrix behavior')
widgets_m, tab_m = gen_tab_behavior(inverse=True)

# Adding inclusions
w_inclusions = widgets.Dropdown(options=list(dict_inclusions.values()), layout={'width': 'max-content'})
button_add_inclusion = widgets.Button(description="Add inclusion")
out1 = widgets.Output() # Displays volume fractions widgets
out2 = widgets.Output() # Displays 'already added inclusion' messages
widgets_f = {} # Contains already added inclusions and their widgets ('name','volume fraction')
buttons_suppress = {} # 'Remove inclusion' buttons and their associated inclusions

button_add_inclusion.on_click(add_inclusion_to_structure)
button_generate_inclusion.on_click(add_inclusion_to_list)
button_generate_inclusion_bis.on_click(add_inclusion_to_list)

# Microstructure generation
b_generate_structure = widgets.Button(description='Generate microstructure', layout={'width': 'max-content'})
# TODO : widget 'valid' qui indique en temps réel si les fractions volumiques choisies sont cohérentes
out3 = widgets.Output()
b_generate_structure.on_click(generate_microstructure)

# Displays microstructure widgets
w_micro = [caption, tab_m, widgets.HBox([w_inclusions, button_add_inclusion, out2]), out1, widgets.HBox([b_generate_structure]),out3]
display(Markdown("## Microstructure generation"))
display(widgets.VBox(w_micro, layout=layout))

def compute_error(microstructure, model, target):
    """
    Compute the least squares error between the target and the homogenized beahviors.
    """
    behavior_h = model.compute_h_behavior(microstructure)
    # For the case of ellipsoid transform C and S in  E, nu, G and K
    if 'K' not in list(behavior_h.keys()):
        behavior_h = Isotropic_behavior(behavior_h)
        
    difference = [target[parameter]-behavior_h[parameter] for parameter in target.keys()]
    return np.linalg.norm(difference)

def grad(target, model, ddl, error_old):
    """
    target: dict, target behavior
    model: Model instance
    microstructure: Microstructure instance
    ddl: dict, contains the optimization problem unknown parameters and their bounds
    error_old: float, error computed for the on-going iteration
    """
    global microstructure
    result = {} # format: {variable function: value of the gradient}
    # Term by term gradient computation
    for function, min_max in ddl.items():
        mini, maxi, pas = min_max # Variable bounds
        pas_i = (maxi - mini)*pas # Common to all the parameters
        changed = function(pas_i, mini, maxi) # Parameter update
        # New error computation
        error_new = compute_error(microstructure, model, target)
        # Parameter gradient
        result[function] = (error_new - error_old)/pas_i
        if changed:
            function(-pas_i, mini, maxi)
    return result

def optimize(target, model, ddl):
    """
    Uses the gradient algorithm to optimize the 'ddl' parameters and reach the target behavior.
    """
    global microstructure
    try:
        pas = list(ddl.values())[0][2] # First variable step, used for all the variables
    except:
        return None # No unknown parameters
    seuil = 10**(-4)
    criteria = seuil + 1 # Criteria initialisation
    max_iterations = 10000
    iteration = 0
    errors = []
    # Error value
    error = compute_error(microstructure, model, target)
    while criteria>seuil and iteration<max_iterations:
        with output_optimization:
            clear_output(wait=True)
            print("Computing...")
        # Gradient computation
        grad_error = grad(target, model, ddl, error)
        #print("iter ", iteration, " error ", error," fi ", microstructure.dict_inclusions[inclusion]," grad ", list(grad_error.values())[0])
        iteration += 1
        # TODO : coder une optimization de alpha
        # Variable update
        for function, minmax in ddl.items():
            mini, maxi = minmax[:2]
            pas_i = pas
            value_incr = -pas_i*grad_error[function]
            function(value_incr, mini, maxi) # Calcul des variables à l'itération suivante
#             print("iter ", iteration, " error ", error," Gm ", microstructure.behavior['G']," grad ", list(grad_error.values())[0]," value_incr ", value_incr)
#            print("iter ", iteration, " error ", error," fi ", microstructure.dict_inclusions," grad ", list(grad_error.values())[0]," value_incr ", value_incr)
#             print("iter ", iteration, " error ", error," value_incr ", value_incr)
        # Error update
        error = compute_error(microstructure, model, target)
        errors.append(error)
        if len(errors)>2:
            criteria = errors[-2] - errors[-1]
    behavior = model.compute_h_behavior(microstructure)
    return microstructure, errors, iteration, behavior

### Model selection
def test_models(b=None):
    """
    Called when a microstructure is generated.
    Tests the implemented models on the generated microstructure and creates a 'valid_models' list of compatible models.
    """
    valid_models = []
    if microstructure == None:
        # Checks whether the microstructure has been generated
        return None
    for Model in list_models:
        model = Model()
        valid = model.check_hypothesis(microstructure)
        if valid:
            # The microstructure fits the model hypothesis
            valid_models.append((model.name, model))
    # Updates the model selection widget
    select_model.options = valid_models

valid_models = [] # List of the microstructure-compatible models: [(model_name, Model)]
select_model = widgets.Dropdown()
test_models()
label = widgets.Label(value="Select a model. Only compatible models will be displayed")
b_compute = widgets.Button(description='Optimize')
output_optimization = widgets.Output()

def optimize_microstructure(b):
    """
    Called when the 'optimize' button is toggled.
    Recovers the selected model, computes the homogenized behavior and displays it.
    """
    model = select_model.value
    output_optimization.clear_output()
    microstructure, errors, iteration, behavior = optimize(target_behavior, model, ddl)
    # For the case of ellipsoid transform C and S in  E, nu, G and K
    if 'K' not in list(behavior.keys()):
        behavior = Isotropic_behavior(behavior)
        
    with output_optimization:
        print("optimized microstructure - {} model".format(model.name))
        print("optimization done with {} iterations".format(iteration))
        print("Target behavior: ", target_behavior)
        print("Actual behavior: ", behavior)
        print("optimized microstructure:")
        print(microstructure)
    # Microstructure reset
    generate_microstructure(None)
    

b_generate_structure.on_click(test_models)
b_compute.on_click(optimize_microstructure)

# Displays optimization widgets 
w_opti = [label, widgets.HBox([select_model, b_compute]), output_optimization]
display(Markdown("## optimization"))
display(widgets.VBox(w_opti, layout=layout))


---